In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dota-2-matches/players.csv
/kaggle/input/dota-2-matches/ability_upgrades.csv
/kaggle/input/dota-2-matches/test_player.csv
/kaggle/input/dota-2-matches/match_outcomes.csv
/kaggle/input/dota-2-matches/teamfights_players.csv
/kaggle/input/dota-2-matches/chat.csv
/kaggle/input/dota-2-matches/purchase_log.csv
/kaggle/input/dota-2-matches/test_labels.csv
/kaggle/input/dota-2-matches/ability_ids.csv
/kaggle/input/dota-2-matches/item_ids.csv
/kaggle/input/dota-2-matches/player_time.csv
/kaggle/input/dota-2-matches/patch_dates.csv
/kaggle/input/dota-2-matches/yasp_sample.json
/kaggle/input/dota-2-matches/match.csv
/kaggle/input/dota-2-matches/player_ratings.csv
/kaggle/input/dota-2-matches/hero_names.csv
/kaggle/input/dota-2-matches/cluster_regions.csv
/kaggle/input/dota-2-matches/teamfights.csv
/kaggle/input/dota-2-matches/objectives.csv


The first question I want to answer is: Can we predict the match outcome based on the aggregates of heroes in the match?

To start we are going to read in the data, and start to combine the CSVs to get information about a single match. 

In [2]:
#Each match has one entry, in this case we are focused on match_id, and radiant_win only as our target.
columnsWeWant = ['match_id','radiant_win']
match = pd.read_csv('/kaggle/input/dota-2-matches/match.csv')
match = match[columnsWeWant]

#We are only intrested in this case on the hero_id and match_id, and player_slot. Note that: Player_slot: 0-4 are Radiant, 128-132 are Dire
columnsWeWant = ['match_id','hero_id','player_slot']
players = pd.read_csv('/kaggle/input/dota-2-matches/players.csv')
players = players[columnsWeWant]

#Change from players_slot to boolean for radiant team or not
players["radiant"] = (players['player_slot'] < 5)

#Groups by match id and radiant boolean, and for each will combine hero_id into a list
combinedHeroID = players.groupby(['match_id','radiant']).hero_id.apply(lambda x: x.tolist())

#We unstack so that we can get the team lists for each match_id into columns instead of a row for each match_id
combinedHeroID = combinedHeroID.unstack()

#We rename the columns to make more sense
combinedHeroID.columns = ['dire_heros','radiant_heros']

#Here we merge the two DataFrames together on the match_id columns, and make the match_id the index
merged = combinedHeroID.merge(match,left_on='match_id',right_on='match_id')
merged = merged.set_index('match_id')

merged

,dire_heros,radiant_heros,radiant_win
match_id,,,
0,"[106, 102, 46, 7, 73]","[86, 51, 83, 11, 67]",True
1,"[73, 22, 5, 67, 106]","[7, 82, 71, 39, 21]",False
2,"[38, 7, 10, 12, 85]","[51, 109, 9, 41, 27]",False
3,"[78, 19, 31, 40, 47]","[50, 44, 32, 26, 39]",False
4,"[101, 100, 22, 67, 21]","[8, 39, 55, 87, 69]",True
...,...,...,...
49995,"[32, 7, 109, 35, 112]","[73, 86, 21, 20, 14]",True
49996,"[36, 1, 112, 60, 71]","[93, 74, 100, 32, 85]",True
49997,"[28, 102, 21, 9, 23]","[100, 68, 75, 39, 44]",True


In [3]:
#Here we start making the Machine Learning Model